In [ ]:
!pip install haystack-ai transformers accelerate bitsandbytes sentence_transformers

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install gradio

In [ ]:
from haystack.components.generators import HuggingFaceLocalGenerator
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
import torch

In [ ]:
import os
from getpass import getpass

model_api_key = os.getenv("Hugging Face API Key: ", None) or getpass("Hugging Face API Key: ")

Hugging Face API Key: ··········


In [ ]:
generator = HuggingFaceLocalGenerator("HuggingFaceH4/zephyr-7b-beta", #mistralai/Mistral-7B-Instruct-v0.1
                                 huggingface_pipeline_kwargs={"device_map":"auto",
                                               "model_kwargs":{"load_in_4bit":True,
                                                "bnb_4bit_use_double_quant":True,
                                                "bnb_4bit_quant_type":"nf4",
                                                "bnb_4bit_compute_dtype":torch.bfloat16}},
                                 generation_kwargs={"max_new_tokens": 150})
generator.warm_up()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
summary_prompt_template = """
Given the following news article, provide a detailed and comprehensive summary that covers key discussions, significant points, outcomes, and the overall impact of the news topic.

News Article:
{{news}}

</s>

Summary:
"""

Builder1 = PromptBuilder(template=summary_prompt_template)

In [ ]:
cl1 = Pipeline()
cl1.add_component("Builder1", Builder1)
cl1.add_component("llm1", generator)
cl1.connect("Builder1.prompt", "llm1.prompt")

🚅 Components
  - Builder1: PromptBuilder
  - llm1: HuggingFaceLocalGenerator
🛤️ Connections
  - Builder1.prompt -> llm1.prompt (str)

In [ ]:
def summarize_news(news_article):
    results = cl1.run({"Builder1": {"news": news_article}})
    summarization_result = results["llm1"]["replies"][0]

    return summarization_result

In [ ]:
news = """Urban Beekeeping Takes Flight in Major Cities

Honeybees are buzzing back into the urban landscape. Once primarily associated with rural areas, beekeeping is gaining popularity in cities around the world. This trend not only provides a sweet treat for beekeepers but also offers a surprising array of benefits for both the environment and urban communities.

The decline of bee populations due to habitat loss and pesticide use has raised concerns about pollination, a crucial process for plant reproduction. By nurturing bee colonies in urban settings, residents can contribute to the health of local ecosystems. These city bees help pollinate flowering plants, from ornamental gardens to community vegetable patches, ensuring a vibrant and diverse urban flora.

But the advantages extend beyond pollination. Beekeeping fosters a sense of connection with nature for city dwellers. Observing and caring for bee colonies can be a rewarding experience, fostering a deeper appreciation for the ecological balance. Additionally, urban beekeeping initiatives can serve as educational tools, teaching residents about the importance of bees and the delicate balance of the environment.

However, keeping bees in cities requires careful consideration. Responsible beekeeping practices are essential to ensure the safety of residents and minimize disturbances. Choosing suitable locations for hives, selecting docile bee breeds, and adhering to local regulations are all crucial aspects of successful urban beekeeping.

Overall, the rise of urban beekeeping represents a positive shift towards a more sustainable and connected urban environment. By providing homes for bees in a world where their natural habitats are under threat, city dwellers can contribute to a healthier ecosystem while reaping the sweet rewards of their efforts. """

In [ ]:
summary = summarize_news(news)

print(summary)


Urban beekeeping, once a rural activity, is gaining popularity in major cities worldwide. This trend not only provides a sweet treat for beekeepers but also offers environmental and community benefits. By nurturing bee colonies in urban settings, residents can contribute to the health of local ecosystems by pollinating flowering plants and fostering a deeper appreciation for the ecological balance. Urban beekeeping initiatives can also serve as educational tools, teaching residents about the importance of bees and the delicate balance of the environment. However, responsible beekeeping practices are essential to ensure the safety of residents and minimize disturbances. Overall, the rise of urban beekeeping represents a positive shift towards a more sustainable and connected urban


In [ ]:
news = """
## AI Art Collaborations Take Center Stage: Humans and Machines Create Together

The art world is witnessing a fascinating trend: the emergence of AI-assisted art.  While some may fear robots replacing artists, the reality is far more collaborative. Artists are increasingly embracing AI tools to push creative boundaries, resulting in innovative and thought-provoking works.

One key application is AI's ability to generate variations based on an artist's initial concept. Imagine feeding an AI a sketch or basic idea, and then receiving an array of different renditions, each with its own style and interpretation. This can spark new ideas and lead the artist in unexpected directions.

AI can also serve as a tool for exploration and experimentation. Artists can use AI-powered algorithms to generate entirely new artistic styles or explore different artistic movements. This allows them to break free from their usual techniques and discover entirely new creative avenues.

However, AI's role isn't to replace the human touch. The true magic lies in the synergy between human creativity and AI's computational power. Artists still retain control over the overall vision, selecting and curating the AI-generated outputs that best align with their artistic intent.

The rise of AI art collaborations has already yielded impressive results. Paintings that seamlessly blend classical techniques with surreal, AI-generated elements are captivating audiences. Interactive installations that respond to user input, powered by AI algorithms, are blurring the lines between art and technology.

While concerns about artistic ownership and authenticity in AI-assisted art exist, the dialogue between artists and AI is opening up new possibilities. This partnership offers exciting opportunities for artistic expression, pushing the boundaries of creativity and offering a glimpse into a future where humans and machines collaborate on the canvas.
"""

In [ ]:
summary = summarize_news(news)
print(summary)



The integration of AI in the art world is not about replacing human creativity but rather enhancing it. AI tools are being used to generate variations based on an artist's initial concept, allowing for new ideas and directions. AI is also being used as a tool for exploration and experimentation, enabling artists to break free from their usual techniques and discover new creative avenues. The role of AI is not to replace human creativity but rather to collaborate with it, with artists still retaining control over the overall vision. The rise of AI art collaborations has already yielded impressive results, with paintings that seamlessly blend classical techniques with surreal, AI-generated elements captivating audiences. Interactive installations that respond to user input


In [ ]:
import gradio as gr

def slow_echo(text, history):
    for i in range(len(text)):
        # time.sleep(0.05)
        ans = summarize_news(text)
        yield "Answer: " + ans

title = "News Summarizer"
description = """
## An AI-powered tool that generates concise summaries of news articles.
"""

demo = gr.Interface(fn=slow_echo, inputs="textbox", outputs="textbox", title=title, description=description)

if __name__ == "__main__":
    demo.launch()


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:924: UserWarning: Expected 2 arguments for function <function slow_echo at 0x7ee705c40dc0>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:928: UserWarning: Expected at least 2 arguments for function <function slow_echo at 0x7ee705c40dc0>, received 1.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://eab00b56a94c5d52c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
